In [14]:
import pandas as pd

# Load dataset
df = pd.read_excel("Data_Label.xlsx")

print("=== 5 Data Pertama ===")
display(df.head())

print("\n=== Info Dataset ===")
df.info()

print("\n=== Cek Nilai Unik Label ===")
print(df['Label'].value_counts())

=== 5 Data Pertama ===


,review,Label
0,Harganya naik menjadi 40k krn liburan. Anak2 b...,Negatif
1,Tempat wisata keluarga yang cocok juga untuk e...,Positif
2,Tempat ini bagus untuk berlibur bersama keluar...,Positif
3,Tempat ini lebih ke mini zoo ya sama spot foto...,Positif
4,"bagus, apik, luas daerah wisatanya, kmdian ban...",Positif



=== Info Dataset ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  250 non-null    object
 1   Label   250 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB

=== Cek Nilai Unik Label ===
Label
Negatif    100
Positif    100
Netral      50
Name: count, dtype: int64


In [15]:
!pip install Sastrawi emoji openpyxl

In [16]:
!pip install Sastrawi

In [17]:
import pandas as pd
import re
import emoji
from nltk.tokenize import wordpunct_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from google.colab import files


In [18]:
with open("stopwords-id.txt", "r", encoding="utf-8") as f:
    stop_words = set(f.read().splitlines())

stop_words.update([
    'aja','ya','deh','dong','sih','nih','loh','lho','toh',
    'pa','ha','kan','dll','dg','org','yg','karna','kalo',
    'nggak','ga','gak','udah','udh','nya','nyaa','krn''jadi', 'dari', 'oke', 'dengan', 'karena'
])

print(f"Jumlah stopword dimuat: {len(stop_words)} kata")

# Tampilkan beberapa baris pertama
df.head()

Jumlah stopword dimuat: 782 kata


,review,Label
0,Harganya naik menjadi 40k krn liburan. Anak2 b...,Negatif
1,Tempat wisata keluarga yang cocok juga untuk e...,Positif
2,Tempat ini bagus untuk berlibur bersama keluar...,Positif
3,Tempat ini lebih ke mini zoo ya sama spot foto...,Positif
4,"bagus, apik, luas daerah wisatanya, kmdian ban...",Positif


In [19]:
normalisasi_dict = {
    # singkatan umum
    'gak': 'tidak', 'ga': 'tidak', 'nggak': 'tidak', 'tdk': 'tidak',
    'yg': 'yang', 'aja': 'saja', 'tp': 'tapi', 'sm': 'sama', 'dr': 'dari',
    'sy': 'saya', 'kmu': 'kamu', 'dgn': 'dengan', 'dg': 'dengan', 'trs': 'terus',
    'udh': 'sudah', 'udah': 'sudah', 'utk': 'untuk', 'bkn': 'bukan',
    'krn': 'karena', 'karna': 'karena', 'pdhl': 'padahal', 'klo': 'kalau',
    'kl': 'kalau', 'dl': 'dulu', 'skrg': 'sekarang', 'sdh': 'sudah',
    'trnyata': 'ternyata', 'hrs': 'harus', 'jd': 'jadi', 'jg': 'juga',
    'org': 'orang', 'orgnya': 'orangnya', 'dll': '', 'pa': 'pak', 'ha': '',

    # kata gaul
    'bgt': 'banget', 'bngt': 'banget', 'btw': 'ngomong-ngomong',
    'mager': 'malas gerak', 'anjir': '', 'anjay': '', 'wkwk': '', 'wkwwk': '',
    'haha': '', 'hehe': '', 'lol': '', 'ngab': '', 'mantappp': 'mantap',

    # typo dan variasi
    'baguus': 'bagus', 'terbaikkk': 'terbaik', 'bgttt': 'banget',
    'mkn': 'makan', 'mknnya': 'makanannya', 'mankap': 'mantap'
}


In [20]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def remove_emoji(text):
    return emoji.replace_emoji(str(text), replace='')

def preprocess_text_steps(text, hapus_angka=True):
    steps = {}
    text = str(text).lower()

    # 1️⃣ Case folding
    steps['case_folding'] = text

    # 2️⃣ Hapus emoji
    text = remove_emoji(text)
    steps['remove_emoji'] = text

    # 3️⃣ Hapus frasa khusus (misal nama tempat)
    text = re.sub(r'\bbhumi\s+merapi\b', '', text)
    steps['remove_phrase'] = text

    # 4️⃣ Hapus huruf berulang
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    steps['remove_repeated_char'] = text

    # 5️⃣ Hapus akhiran -nya dan kata 'nya' terpisah
    text = re.sub(r'\b(\w+)nya\b', r'\1', text)
    text = re.sub(r'\bnya\b', '', text)
    steps['remove_suffix_nya'] = text

    # 6️⃣ Hapus angka dan satuannya (30k, 15rb, 2jam, dst.)
    if hapus_angka:
        text = re.sub(r'\b\d+\s?(k|rb|jt|jam|menit|x)?\b', '', text)
    steps['remove_numbers'] = text

    # 7️⃣ Hapus simbol / karakter non-huruf
    text = re.sub(r'[^a-z\s]', ' ', text)
    steps['remove_nonalpha'] = text

    # 8️⃣ Tokenisasi
    tokens = wordpunct_tokenize(text)
    steps['tokenization'] = tokens

    # 9️⃣ Normalisasi (singkatan + kata gaul)
    tokens = [normalisasi_dict[w] if w in normalisasi_dict else w for w in tokens]
    tokens = [w for w in tokens if w != '']
    steps['normalization'] = tokens

    # 🔟 Stopword Removal (setelah normalisasi)
    tokens = [w for w in tokens if w not in stop_words and len(w) > 1]
    steps['stopword_removal'] = tokens

    # 1️⃣1️⃣ Stemming
    tokens = [stemmer.stem(w) for w in tokens]
    steps['stemming'] = tokens

    # 1️⃣2️⃣ Gabungkan kembali
    cleaned_text = ' '.join(tokens)
    steps['final_text'] = cleaned_text.strip()

    return steps


In [21]:
def preprocess_final(text):
    return preprocess_text_steps(text)['final_text']

df['cleaned_review'] = df['review'].apply(preprocess_final)

In [22]:
from sklearn.model_selection import train_test_split

X = df['cleaned_review']
y = df['Label']

# 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Jumlah data train:", len(X_train))
print("Jumlah data test :", len(X_test))


Jumlah data train: 200
Jumlah data test : 50


**EXTRAKSI FITUR TF-IDF**

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),  # unigram + bigram
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

X_train_tfidf.shape, X_test_tfidf.shape


((200, 3293), (50, 3293))

**Train Model Pertama: SVM**

In [24]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

svm_clf = LinearSVC()
svm_clf.fit(X_train_tfidf, y_train)

y_pred = svm_clf.predict(X_test_tfidf)

print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Akurasi: 0.72

Classification Report:

              precision    recall  f1-score   support

     Negatif       0.79      0.75      0.77        20
      Netral       0.50      0.20      0.29        10
     Positif       0.70      0.95      0.81        20

    accuracy                           0.72        50
   macro avg       0.66      0.63      0.62        50
weighted avg       0.70      0.72      0.69        50



**PERCOBAAN KEDUA: Naive Bayes**

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

nb_clf = MultinomialNB()
nb_clf.fit(X_train_tfidf, y_train)

y_pred_nb = nb_clf.predict(X_test_tfidf)

print("=== HASIL TF-IDF + NAIVE BAYES ===")
print("Akurasi:", accuracy_score(y_test, y_pred_nb))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_nb))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_nb))


=== HASIL TF-IDF + NAIVE BAYES ===
Akurasi: 0.64

Classification Report:

              precision    recall  f1-score   support

     Negatif       0.74      0.70      0.72        20
      Netral       0.00      0.00      0.00        10
     Positif       0.58      0.90      0.71        20

    accuracy                           0.64        50
   macro avg       0.44      0.53      0.47        50
weighted avg       0.53      0.64      0.57        50


Confusion Matrix:
[[14  0  6]
 [ 3  0  7]
 [ 2  0 18]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**PERCOBAAN KETIGA: LSTM**

In [26]:
import numpy as np

X_train_dense = X_train_tfidf.toarray()
X_test_dense = X_test_tfidf.toarray()

print(X_train_dense.shape, X_test_dense.shape)


(200, 3293) (50, 3293)


In [27]:
X_train_lstm = X_train_dense.reshape((X_train_dense.shape[0], 1, X_train_dense.shape[1]))
X_test_lstm = X_test_dense.reshape((X_test_dense.shape[0], 1, X_test_dense.shape[1]))

X_train_lstm.shape, X_test_lstm.shape


((200, 1, 3293), (50, 1, 3293))

In [28]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

num_classes = len(encoder.classes_)
print("Jumlah kelas:", num_classes)


Jumlah kelas: 3


In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(128, input_shape=(1, X_train_dense.shape[1]), return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │     1,752,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,760,515 (6.72 MB)

 Trainable params: 1,760,515 (6.72 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
history = model.fit(
    X_train_lstm, y_train_enc,
    epochs=8,
    batch_size=16,
    validation_split=0.2,
    verbose=1
)


Epoch 1/8
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - accuracy: 0.4355 - loss: 1.0965 - val_accuracy: 0.4750 - val_loss: 1.0916
Epoch 2/8
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6935 - loss: 1.0784 - val_accuracy: 0.4500 - val_loss: 1.0836
Epoch 3/8
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.7445 - loss: 1.0484 - val_accuracy: 0.4500 - val_loss: 1.0765
Epoch 4/8
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8109 - loss: 0.9911 - val_accuracy: 0.5000 - val_loss: 1.0569
Epoch 5/8
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7948 - loss: 0.9262 - val_accuracy: 0.5000 - val_loss: 1.0425
Epoch 6/8
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8441 - loss: 0.7779 - val_accuracy: 0.5250 - val_loss: 1.0111
Epoch 7/8
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9388 - loss: 0.6101 - val_accuracy: 0.5000 - val_loss: 0.9875
Epoch 8/8
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9904 - loss: 0.3959 - val_accuracy: 0.5250 - val_loss:

In [31]:
from sklearn.metrics import classification_report, accuracy_score

y_pred_prob = model.predict(X_test_lstm)
y_pred_class = np.argmax(y_pred_prob, axis=1)

print("Akurasi:", accuracy_score(y_test_enc, y_pred_class))
print("\nClassification Report:")
print(classification_report(y_test_enc, y_pred_class, target_names=encoder.classes_))


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 370ms/step
Akurasi: 0.66

Classification Report:
              precision    recall  f1-score   support

     Negatif       0.62      0.90      0.73        20
      Netral       0.00      0.00      0.00        10
     Positif       0.71      0.75      0.73        20

    accuracy                           0.66        50
   macro avg       0.44      0.55      0.49        50
weighted avg       0.53      0.66      0.59        50



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**PERCOBAAN KE EMPAT: BERT Transformer**

In [34]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

num_labels = len(le.classes_)
num_labels


3

In [35]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': enc['input_ids'].squeeze(),
            'attention_mask': enc['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.texts)


In [36]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

model = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1",
    num_labels=num_labels
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
train_ds = TextDataset(X_train.tolist(), y_train_enc.tolist(), tokenizer)
test_ds  = TextDataset(X_test.tolist(), y_test_enc.tolist(), tokenizer)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=8)


In [39]:
from torch.optim import AdamW
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 3

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")


Epoch 1/3, Loss: 27.3290
Epoch 2/3, Loss: 23.2144
Epoch 3/3, Loss: 15.0474


In [40]:
from sklearn.metrics import accuracy_score, classification_report

model.eval()
preds = []
targets = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        pred = torch.argmax(F.softmax(logits, dim=1), dim=1)

        preds.extend(pred.cpu().numpy())
        targets.extend(batch['labels'].numpy())

print("Akurasi:", accuracy_score(targets, preds))
print("\nClassification Report:")
print(classification_report(targets, preds, target_names=le.classes_))


Akurasi: 0.68

Classification Report:
              precision    recall  f1-score   support

     Negatif       0.71      0.85      0.77        20
      Netral       0.00      0.00      0.00        10
     Positif       0.65      0.85      0.74        20

    accuracy                           0.68        50
   macro avg       0.45      0.57      0.50        50
weighted avg       0.54      0.68      0.60        50



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
